In [1]:
from bs4 import BeautifulSoup, NavigableString
import re

In [2]:
with open('wfg2.html', 'r') as f:
    html = ''.join(f.readlines())

In [3]:
soup = BeautifulSoup(html, 'lxml')

In [4]:
# largest element
main = max([*soup.find_all('blockquote')], key=len)
print(main.prettify())

<blockquote>
 <dd>
  <i>
   Estragon's boots front center, heels together, toes splayed.
   <font color="#ffffff" size="0">
    #
   </font>
   <blockquote>
    <img border="0" height="289" src="SanchezSt3.jpg" width="371"/>
   </blockquote>
  </i>
 </dd>
 <dd>
  <i>
   Lucky's hat at same place.
  </i>
 </dd>
 <dd>
  <i>
   The tree has four or five leaves.
  </i>
 </dd>
 <dd>
  <i>
   Enter Vladimir agitatedly. He halts and looks long at the tree, then suddenly begins to move feverishly about the stage. He halts before the boots, picks one up, examines it, sniffs it, manifests disgust, puts it back carefully. Comes and goes. Halts extreme right and gazes into distance off, shading his eyes with his hand. Comes and goes. Halts extreme left, as before. Comes and goes. Halts suddenly and begins to sing loudly.
  </i>
 </dd>
 <dt>
  <p>
   <b>
    VLADIMIR:
   </b>
  </p>
 </dt>
 <dd>
  <blockquote>
   <blockquote>
    A dog came in–
   </blockquote>
  </blockquote>
 </dd>
 <dd>
  <i>
  

In [5]:
with open('wfg2.txt', 'w', encoding='utf-8') as f:
    current_text = ''
    current_speaker = ''
    unwanted_tags = ['i', 'blockquote', 'ul']
    for tag in main:
        if tag.name == 'dt':
            current_text = ''
            current_speaker = tag.b.text

            # italic is not conversation
            temp_tag = tag.find_next_sibling('dd', recursive=False)
            while temp_tag and temp_tag.name == 'dd':
                if not temp_tag.find_all('div') and not temp_tag.find_all('font', {'face': 'arial'}):
                    for inner_tag in temp_tag:
                        if type(inner_tag) is NavigableString:
                            current_text += inner_tag
                        else:
                            if all([inner_tag.name != tag_name for tag_name in unwanted_tags]) and all([not inner_tag.find_all(tag_name) for tag_name in unwanted_tags]):
                                current_text += inner_tag.text
                temp_tag = temp_tag.next_sibling

            if tag.find('img'):
                # directly creat an entry
                current_text = ''.join([inner_tag if type(inner_tag) is NavigableString else inner_tag.text for inner_tag in tag if inner_tag.name != 'b'])

            current_text = current_text.replace('\n', '').replace('. . .', '…').replace('#', '')
            # clear anything inside bracket
            current_text = re.sub(r'\((.*?)\)', '', current_text)
            # clear remaining brackets
            current_text = re.sub(r'[\(\)]', '', current_text)
            current_text = re.sub(r' +', ' ', current_text)
            current_text = current_text.replace('… …', '…').strip()
            if current_text:
                f.write(current_speaker + ' ' + current_text + '\n')